In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import matplotlib.pyplot as plt
import numpy as np
import math


In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

train_dir = os.path.join('MNIST_FILE')
val_dir = os.path.join('MNIST_FILE/')

train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')

# number of classes
K = 4



Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [8]:
input_data = tf.keras.Input(shape=(224, 224, 3), dtype='float32', name='input')  # 224*224로 샘플링, 채널3

def conv1_layer(x):
    x = tf.keras.layers.ZeroPadding2D(padding = (3,3))(x)  # 이미지 사이즈 맞추기
    x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2,2))(x)  # 입력값, 7*7,64,stride=2
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.ZeroPadding2D(padding=(1,1))(x)

    return x


def conv2_layer(x):
    x = tf.keras.layers.MaxPooling2D((3, 3), 2)(x)  # maxpooling 3*3, stride=2

    shortcut = x

    for i in range(3):
        if i == 0:  # 이전 stage에서 가져온 텐서의 dimension을 증가 시켜야 함
            x = tf.keras.layers.Conv2D(64, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)  # 배치 정규화
            x = tf.keras.layers.Activation('relu')(x)  # relu

            x = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(x)  # 3*3 필터 사용할 때 padding = same
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(256, (1, 1))(x)
            shortcut = tf.keras.layers.Conv2D(256, (1, 1))(shortcut)
            x = tf.keras.layers.BatchNormalization()(x)
            shortcut = tf.keras.layers.BatchNormalization()(shortcut)

            x = tf.keras.layers.Add()([x, shortcut])    # 더해주기
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x
        else:
            x = tf.keras.layers.Conv2D(64, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(256, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x

    return x


def conv3_layer(x):
    shortcut = x

    for i in range(4):
        if i == 0:          # 이전 stage에서 가져온 텐서의 dimension을 증가 시켜야 함
            x = tf.keras.layers.Conv2D(128, (1, 1), strides=(2,2))(x)  # 논문 3.3 1*1 convolutions 는 stride2
            x = tf.keras.layers.BatchNormalization()(x)                #배치
            x = tf.keras.layers.Activation('relu')(x)                  #

            x = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(512, (1, 1))(x)  # input 의 차원이 1x1 convolution과 stride=2에 의해 바뀌게 된다.
            shortcut = tf.keras.layers.Conv2D(512, (1, 1), strides=(2,2))(shortcut)  # 그러므로 skip connection도 차원 변화를 처리해줘야 한다.
            x = tf.keras.layers.BatchNormalization()(x)
            shortcut = tf.keras.layers.BatchNormalization()(shortcut)

            x = tf.keras.layers.Add()([x, shortcut])   #residual connection
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x
        else:
            x = tf.keras.layers.Conv2D(128, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(512, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x

    return x


def conv4_layer(x):
    shortcut = x

    for i in range(6):
        if i == 0:
            x = tf.keras.layers.Conv2D(256, (1, 1), strides=(2,2))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(1024, (1, 1))(x)
            shortcut = tf.keras.layers.Conv2D(1024, (1, 1), strides=(2,2))(shortcut)
            x = tf.keras.layers.BatchNormalization()(x)
            shortcut = tf.keras.layers.BatchNormalization()(shortcut)

            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x

        else:
            x = tf.keras.layers.Conv2D(256, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')

            x = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)

            x = tf.keras.layers.Conv2D(1024, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x
    return x


def conv5_layer(x):
    shortcut = x

    for i in range(3):
        if i == 0:
            x = tf.keras.layers.Conv2D(512, (1, 1), strides=(2,2))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')

            x = tf.keras.layers.Conv2D(512, (3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')

            x = tf.keras.layers.Conv2D(2048, (1, 1))(x)
            shortcut = tf.keras.layers.Conv2D(2048, (1, 1), strides=(2,2))(shortcut)
            x = tf.keras.layers.BatchNormalization(x)
            shortcut = tf.keras.layers.BatchNormalization()(shortcut)

            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')

            shortcut = x
        else:
            x = tf.keras.layers.Conv2D(512, (1, 1))(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')

            x = tf.keras.layers.Conv2D(512, (3, 3), padding='same')
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')

            x = tf.keras.layers.Conv2D(2048, (1, 1), strides=(1,1))(x)
            x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.Add()([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)

            shortcut = x
    return x


x = conv1_layer(input_data)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
output_tensor = tf.keras.layers.Dense(K, activation='softmax')(x)

resnet50 = tf.keras.Model(input_data, output_tensor)
resnet50.summary()

TypeError: Inputs to a layer should be tensors. Got: <tensorflow.python.keras.layers.core.Activation object at 0x00000296106EBA90>